In [1]:
## Fourier Series Regression Experiment
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../lib")))

from data_load import load_data
from validation import rmse_validation, mae_validation, r2_validation

In [2]:
# Load all data and define constants
date_start = pd.to_datetime("2025-02-17 12:00:00 AM")
date_end = pd.to_datetime("2025-02-21 12:00:00 AM")
T = 24 * 60

initial_guess = [50, 10, 10, 5, 5]
garages = [x + " Garage" for x in ["North", "South", "West", "South Campus"]]

all_data = load_data(date_start, date_end)

In [3]:
# Define Fourier series model (order 2 example)
def fourier_model(x, a0, a1, b1, a2, b2):
    return a0 + a1 * np.cos(2 * np.pi * x / T) + b1 * np.sin(2 * np.pi * x / T) + \
           a2 * np.cos(4 * np.pi * x / T) + b2 * np.sin(4 * np.pi * x / T)

In [4]:
def fourier_fit(data):
    # replace time stamps with time from date_start in minutes instead
    # Extract x and y
    x_data = (data['timestamp'] - date_start).dt.total_seconds() / 60
    y_data = data['fullness']
    
    # fit curve with params
    params, _ = curve_fit(fourier_model, x_data, y_data, p0=initial_guess)
    print("fitted sine function using coefficients:")
    print(params)
    
    # generate x points for sin curves
    x_end = int(x_data.tail(1).values[0])
    all_x_vals = np.linspace(0, x_end, num=1000)
    
    return x_data, all_x_vals, y_data, fourier_model(x_data, *params), fourier_model(all_x_vals, *initial_guess)


def model_plot(x_data, all_x_data, y_data, fit, guess_fit):
    plt.scatter(x_data, y_data, color='blue', alpha=0.5, label='raw data')
    
    plt.plot(x_data, y_data, color='blue', linestyle='--')
    plt.plot(x_data, fit, color='red', label='fitted curve')
    
    # plt.plot(all_x_data, guess_fit, color='orange', label='init guess curve')
    plt.xlabel('Time (minutes)')
    plt.ylabel('Fullness')
    plt.title('Fourier Regression')
    plt.show()
 

In [5]:
north_data = all_data[all_data['garage name'] == garages[0]]

north_fit_params = fourier_fit(north_data)

NameError: name 'fourier_model' is not defined

In [ ]:
model_plot(*north_fit_params)

In [ ]:
south_data = all_data[all_data['garage name'] == garages[1]]

south_fit_params = fourier_fit(south_data)

In [ ]:
model_plot(*south_fit_params)

In [ ]:
west_data = all_data[all_data['garage name'] == garages[2]]

west_fit_params = fourier_fit(west_data)

In [ ]:
model_plot(*west_fit_params)

In [ ]:
south_campus_data = all_data[all_data['garage name'] == garages[3]]

south_campus_fit_params = fourier_fit(south_campus_data)

In [ ]:
model_plot(*south_campus_fit_params)

In [5]:
north_actual_pred = north_fit_params[2], north_fit_params[3]
south_actual_pred = south_fit_params[2], south_fit_params[3]
west_actual_pred = west_fit_params[2], west_fit_params[3]
south_campus_pred = south_campus_fit_params[2], south_campus_fit_params[3]

# pass in raw data and predicted sinusoidal regression values 
north_rmse = rmse_validation(*north_actual_pred)
south_rmse = rmse_validation(*south_actual_pred)
west_rmse = rmse_validation(*west_actual_pred)
south_campus_rmse = rmse_validation(*south_campus_pred)

north_mae = mae_validation(*north_actual_pred)
south_mae = mae_validation(*south_actual_pred) 
west_mae = mae_validation(*west_actual_pred) 
south_campus_mae = mae_validation(*south_campus_pred) 

north_r2 = r2_validation(*north_actual_pred)
south_r2 = r2_validation(*south_actual_pred)
west_r2 = r2_validation(*west_actual_pred)
south_campus_r2 = r2_validation(*south_campus_pred)

print("rmse cals: ")
print(f'north rmse: {north_rmse}')
print(f'south rmse: {south_rmse}')
print(f'west rmse: {west_rmse}')
print(f'south campus rmse: {south_campus_rmse}')
print("---------------")
print("mae calcs: ")
print(f'north rmse: {north_mae}')
print(f'south rmse: {south_mae}')
print(f'west rmse: {west_mae}')
print(f'south campus rmse: {south_campus_mae}')
print("----------------")
print("r2 calcs: ")
print(f'north rmse: {north_r2}')
print(f'south rmse: {south_r2}')
print(f'west rmse: {west_r2}')
print(f'south campus rmse: {south_campus_r2}')

RMSE calculations:
North RMSE: 6.247927501467318
South RMSE: 6.247927501467318
West RMSE: 6.247927501467318
South Campus RMSE: 6.247927501467318
---------------
MAE calculations:
North MAE: 4.674134228325856
South MAE: 4.674134228325856
West MAE: 4.674134228325856
South Campus MAE: 4.674134228325856
----------------
R² calculations:
North R²: 0.9650704864878141
South R²: 0.9650704864878141
West R²: 0.9650704864878141
South Campus R²: 0.9650704864878141
